In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os
import re

class TownUrlExtractor:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
    
    def fetch_html(self, url):
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.text
        except:
            return None
    
    def extract_towns(self, html_content):
        soup = BeautifulSoup(html_content, 'html.parser')
        towns = []
        
        panels = soup.find('div', class_='c-tabs__panels list-balloon__towns')
        if panels:
            links = panels.find_all('a', class_='c-link-arrow')
            for link in links:
                href = link.get('href')
                span = link.find('span')
                town_name = span.text.strip() if span else ''
                if href and town_name:
                    towns.append({'town_name': town_name, 'url': href})
        
        return towns
    
    def process_dataframe(self, df, url_column=None):
        """
        DataFrameを処理してタウン情報を抽出する
        
        Args:
            df (pd.DataFrame): 処理対象のDataFrame
            url_column (str, optional): URLが含まれるカラム名
        
        Returns:
            list: 抽出されたタウン情報のリスト
        """
        # URLカラムの自動検出
        if url_column is None:
            for col in df.columns:
                if 'url' in col.lower() or 'link' in col.lower():
                    url_column = col
                    break
            if url_column is None:
                url_column = df.columns[0]
        
        all_towns = []
        
        for _, row in df.iterrows():
            url = row[url_column]
            if pd.notna(url):  # NaNチェックを追加
                html_content = self.fetch_html(url)
                if html_content:
                    towns = self.extract_towns(html_content)
                    all_towns.extend(towns)
        
        return all_towns
    
    def save_results(self, towns, output_file='extracted_towns.csv'):
        """
        結果をCSVファイルに保存する
        
        Args:
            towns (list): タウン情報のリスト
            output_file (str): 出力ファイル名
        
        Returns:
            str: 出力ファイル名
        """
        pd.DataFrame(towns).to_csv(output_file, index=False, encoding='utf-8')
        return output_file

class TabelogStoreScraper:
    """
    食べログから店舗詳細情報を取得するクラス
    """
    def __init__(self, url_list=None, max_num=60):
        """
        店舗詳細情報のスクレイピングを開始する
        
        Args:
            url_list (list): URLのリスト。文字列のリストまたは辞書のリスト。Noneの場合は元のデフォルトURL
            max_num (int): 各URLで検索するページ上限数
        """
        # 取得したデータ一覧の保存場所
        self.RESULT_PATH = 'out'
        # 取得したデータ一覧名
        self.STORE_RESULT_NAME = 'store_details.csv'
        # 店舗詳細データフレームの列名（地図情報を追加）
        self.STORE_COLUMNS = ['store_id', 'store_name', 'genre', 'rate', 'review_cnt', 'store_url',
                             'address', 'latitude', 'longitude', 'business_hours', 'reserve', 
                             'dinner_budget', 'lunch_budget', 'opened']
        # 検索するページ上限数(1,200[20*60]件以降は表示不可)
        self.max_num = max_num + 1 
        self.shop_id = ''
        self.id_num = 1
        self.store_df = pd.DataFrame(columns=self.STORE_COLUMNS)
        
        # URLリストの設定
        if url_list:
            # 辞書のリストの場合はURLを抽出
            if isinstance(url_list[0], dict):
                self.url_list = [item['url'] for item in url_list]
            else:
                self.url_list = url_list
        else:
            # デフォルトのURL（元のコードと同じ）
            self.url_list = ['https://tabelog.com/tokyo/C13105/C36312/rstLst/']

        self._scrape_store_details()

    def _scrape_store_details(self):
        """
        店舗詳細情報をスクレイピングするメソッド
        """
        try:
            first_time = time.time()
            
            # 各URLに対して処理を実行
            for base_url in self.url_list:
                for page_num in range(1, self.max_num):
                    # ページ番号をURLに追加
                    if base_url.endswith('/'):
                        page_url = base_url + str(page_num) + '/?Srt=D&SrtT=rt&sort_mode=1'
                    else:
                        page_url = base_url + '/' + str(page_num) + '/?Srt=D&SrtT=rt&sort_mode=1'

                    response = self.connect_url(page_url)
                    # HTML解析用の変数
                    soup = BeautifulSoup(response.text, 'lxml')
                    # 現ページの表示件数
                    count_num_element = soup.find('span', class_='c-page-count__num')
                    count_num = count_num_element.get_text(strip=True) if count_num_element else '0'
                    # 0件の場合はbreak
                    if count_num == '0':
                        break
                    
                    # 一覧ページから以下の店舗情報を取得するリスト化 
                    # ジャンルの取得
                    genre_info = [genre.text.rstrip() for genre in soup.find_all('div', class_='list-rst__area-genre cpy-area-genre')]
                    # 評価点の取得
                    rate_list = [float(rate.text) for rate in soup.find_all('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val')]
                    # 口コミ件数の取得
                    count_list = [count.text for count in soup.find_all('em', class_='list-rst__rvw-count-num cpy-review-count')]
                    # 店名の取得
                    raw_data = soup.find_all('a', class_='list-rst__rst-name-target cpy-rst-name js-ranking-num')
                    shop_list = [name.text for name in raw_data]
                    shop_url_list = [shop_url.get('href') for shop_url in raw_data]

                    # 最大のリスト長を取得
                    max_length = max(len(shop_list), len(genre_info), len(rate_list), len(count_list), len(shop_url_list))
                    
                    # 各リストを最大長に合わせて空文字で埋める
                    while len(genre_info) < max_length:
                        genre_info.append("")
                    while len(rate_list) < max_length:
                        rate_list.append(0.0)
                    while len(count_list) < max_length:
                        count_list.append("0")
                    while len(shop_list) < max_length:
                        shop_list.append("")
                    while len(shop_url_list) < max_length:
                        shop_url_list.append("")

                    for i in range(min(20, len(shop_list))):
                        # 店名が空の場合はスキップ
                        if not shop_list[i]:
                            continue
                            
                        # URLから店舗IDを抽出
                        if shop_url_list[i]:
                            # URLの最後の数字部分を店舗IDとして抽出
                            # 例: https://tabelog.com/ishikawa/A1702/A170203/17000700/ -> 17000700
                            url_parts = shop_url_list[i].strip('/').split('/')
                            for part in reversed(url_parts):
                                if part.isdigit() and len(part) >= 7:  # 7桁以上の数字を店舗IDとして判定
                                    self.shop_id = part
                                    break
                            else:
                                # 数字が見つからない場合は連番
                                self.shop_id = str(self.id_num).zfill(5)
                        else:
                            # URLがない場合は連番
                            self.shop_id = str(self.id_num).zfill(5)
                        
                        # ジャンル情報の処理（空でない場合のみ分割）
                        if genre_info[i]:
                            genre_parts = genre_info[i].split("/")
                            genre = genre_parts[-1].strip() if genre_parts else ""
                        else:
                            genre = ""

                        start_time = time.time()

                        # 店舗詳細ページから詳細情報を取得（URLが存在する場合のみ）
                        if shop_url_list[i]:
                            shop_detail_url = shop_url_list[i]
                            print(f"店舗詳細URL: {shop_detail_url}")
                            detail_response = self.connect_url(shop_detail_url)
                            detail_soup = BeautifulSoup(detail_response.text, 'lxml')
                            
                            # 各種詳細情報を取得
                            address = self.extract_address(detail_soup)
                            business_hours = self.extract_business_hours(detail_soup)
                            reserve = self.extract_reserve(detail_soup)
                            budget_info = self.extract_budget_info(detail_soup)
                            dinner_budget = budget_info.get('dinner', '')
                            lunch_budget = budget_info.get('lunch', '')
                            opened = self.extract_opened(detail_soup)
                            # 地図情報を取得
                            map_info = self.extract_map_info(detail_soup)
                            latitude = map_info.get('latitude', '')
                            longitude = map_info.get('longitude', '')
                        else:
                            # URLがない場合は詳細情報は空
                            shop_detail_url = ""
                            address = ""
                            business_hours = ""
                            reserve = ""
                            dinner_budget = ""
                            lunch_budget = ""
                            opened = ""
                            map_link = ""
                            latitude = ""
                            longitude = ""

                        # データフレームに店舗詳細情報を追加（地図情報も含める）
                        self.add_store_df(shop_list[i], genre, rate_list[i], 
                                        count_list[i], shop_detail_url, address, latitude, longitude,
                                        business_hours, reserve, dinner_budget, lunch_budget, opened)
                        
                        # 結果をログ出力
                        self.write_store_result(shop_list[i], genre_info[i].strip() if genre_info[i] else "", 
                                              rate_list[i], count_list[i])
                        process_time = time.time() - start_time
                        print('店舗ID: {}, {}件目完了：　処理時間：{:.3f}秒'.format(self.shop_id, self.id_num, process_time))
                        self.id_num += 1

        except requests.exceptions.HTTPError as e:
            print(e)
        except requests.exceptions.ConnectTimeout as e:
            print(e)
        finally:
            end_time = time.time() - first_time
            print('店舗詳細取得終了、処理時間：{:.3f}秒'.format(end_time))

    def extract_address(self, soup):
        """BeautifulSoupオブジェクトから住所情報を抽出する関数"""
        try:
            address_element = soup.find('p', class_='rstinfo-table__address')
            if not address_element:
                return ""
            full_address = address_element.get_text(separator="/", strip=True)
            return full_address
        except Exception as e:
            print(f"住所抽出エラー: {e}")
            return ""
            
    def extract_business_hours(self, soup):
        """BeautifulSoupオブジェクトから営業時間情報を抽出する関数"""
        try:
            business_lists = soup.find_all('ul', class_='rstinfo-table__business-list')
            if not business_lists:
                return ""
            
            all_text_parts = []
            for ul in business_lists:
                text = ul.get_text(separator='/', strip=True)
                if text:
                    text = re.sub(r'\s+', ' ', text)
                    text = text.strip()
                    if text:
                        all_text_parts.append(text)
            
            result = ' '.join(all_text_parts)
            result = re.sub(r'\s+', ' ', result)
            result = result.strip()
            return result
        except Exception as e:
            print(f"営業時間抽出エラー: {e}")
            return ""

    def extract_reserve(self, soup):
        """BeautifulSoupオブジェクトから予約可否情報を抽出する関数"""
        try:
            reserve_element = soup.find('p', class_='rstinfo-table__reserve-status')
            if not reserve_element:
                return ""
            reserve = reserve_element.get_text(strip=True)
            return reserve
        except Exception as e:
            print(f"予約可否抽出エラー: {e}")
            return ""
    
    def extract_budget_info(self, soup):
        """BeautifulSoupオブジェクトから予算情報を抽出する関数"""
        try:
            budget_div = soup.find('div', class_='rdheader-budget')
            if not budget_div:
                return {'dinner': '', 'lunch': ''}
            
            budget_info = {'dinner': '', 'lunch': ''}
            budget_items = budget_div.find_all('p', class_='c-rating-v3')
            
            for item in budget_items:
                time_icon = item.find('i', class_=re.compile(r'c-rating-v3__time'))
                if time_icon:
                    if 'c-rating-v3__time--dinner' in time_icon.get('class', []):
                        time_type = 'dinner'
                    elif 'c-rating-v3__time--lunch' in time_icon.get('class', []):
                        time_type = 'lunch'
                    else:
                        continue
                    
                    price_link = item.find('a', class_='rdheader-budget__price-target')
                    if price_link:
                        budget_info[time_type] = price_link.get_text(strip=True)
                    else:
                        price_span = item.find('span', class_='c-rating-v3__val')
                        if price_span:
                            budget_info[time_type] = price_span.get_text(strip=True)
            
            return budget_info
        except Exception as e:
            print(f"予算情報抽出エラー: {e}")
            return {'dinner': '', 'lunch': ''}
    
    def extract_opened(self, soup):
        """BeautifulSoupオブジェクトからオープン日情報を抽出する関数"""
        try:
            opened_element = soup.find('p', class_='rstinfo-opened-date')
            if not opened_element:
                return ""
            opened = opened_element.get_text(strip=True)
            return opened
        except Exception as e:
            print(f"オープン日抽出エラー: {e}")
            return ""

    def extract_map_info(self, soup):
        """BeautifulSoupオブジェクトから座標情報を抽出する関数"""
        try:
            map_info = {'latitude': '', 'longitude': ''}
            
            # 地図のwrapperを探す
            map_wrap = soup.find('div', class_='rstinfo-table__map-wrap')
            if not map_wrap:
                return map_info
            
            # Google Maps静的地図のimgタグから座標を抽出
            img_element = map_wrap.find('img', class_='rstinfo-table__map-image')
            if img_element:
                # data-lazy-srcまたはsrcからGoogle Maps APIのURLを取得
                map_url = img_element.get('data-lazy-src') or img_element.get('src')
                if map_url:
                    # centerパラメータから座標を抽出
                    # URL例: https://maps.googleapis.com/maps/api/staticmap?...&center=36.569571136359684,136.6679997943676&...
                    center_match = re.search(r'center=([0-9.-]+),([0-9.-]+)', map_url)
                    if center_match:
                        map_info['latitude'] = center_match.group(1)
                        map_info['longitude'] = center_match.group(2)
                    
                    # markersパラメータからも座標を取得（より正確な場合がある）
                    markers_match = re.search(r'markers=color:red\|([0-9.-]+),([0-9.-]+)', map_url)
                    if markers_match:
                        map_info['latitude'] = markers_match.group(1)
                        map_info['longitude'] = markers_match.group(2)
            
            return map_info
        except Exception as e:
            print(f"地図情報抽出エラー: {e}")
            return {'latitude': '', 'longitude': ''}

    def connect_url(self, target_url):
        """対象のURLにアクセスする関数"""
        data = requests.get(target_url, timeout=10)
        data.encoding = data.apparent_encoding
        time.sleep(2)  # アクセス過多を避けるため、2秒スリープ

        if data.status_code == requests.codes.ok:
            return data
        else:
            data.raise_for_status()

    def write_store_result(self, name, genre, rate, count):
        """店舗詳細情報のログを出力する関数"""
        if not os.path.exists(self.RESULT_PATH):
            os.makedirs(self.RESULT_PATH)
        
        file_path = os.path.join(self.RESULT_PATH, self.STORE_RESULT_NAME)
        with open(file_path, mode='a', encoding='utf-8') as f:
            f.write('店舗ID: {}, {}, {}, 評価：{}, 口コミ：{}件\n'.format(self.shop_id, name, genre, rate, count))

    def add_store_df(self, name, genre, rate, count, store_url, address, latitude, longitude,
                     business_hours, reserve, dinner_budget, lunch_budget, opened):
        """店舗詳細データフレームに新しい行を追加する関数（座標情報も含める）"""
        new_row = {
            'store_id': self.shop_id,
            'store_name': name,
            'genre': genre,
            'rate': rate,
            'review_cnt': count,
            'store_url': store_url,
            'address': address,
            'latitude': latitude,
            'longitude': longitude,
            'business_hours': business_hours,
            'reserve': reserve,
            'dinner_budget': dinner_budget,
            'lunch_budget': lunch_budget,
            'opened': opened
        }
        new_df = pd.DataFrame([new_row])
        self.store_df = pd.concat([self.store_df, new_df], ignore_index=True)

class TabelogReviewScraper:
    """
    食べログから口コミ情報を取得するクラス
    店舗詳細URLのリストを受け取って口コミを取得する
    """
    def __init__(self, store_urls_data, max_num=60, output_individual_csv=True, output_combined_csv=True):
        """
        店舗URLリストを受け取って口コミスクレイピングを開始する
        
        Args:
            store_urls_data: 店舗情報を含むDataFrameまたは辞書のリスト
                           必要な列: store_id, store_name, store_url, genre, rate, review_cnt
            output_individual_csv: 店舗ごとの個別CSVを出力するか（デフォルト: True）
            output_combined_csv: 全店舗統合CSVを出力するか（デフォルト: True）
        """
        # 取得したデータ一覧の保存場所
        self.RESULT_PATH = 'out'
        # 店舗ごとの口コミCSV保存場所
        self.INDIVIDUAL_CSV_PATH = os.path.join(self.RESULT_PATH, 'reviews_by_store')
        # 取得したデータ一覧名
        self.REVIEW_RESULT_NAME = 'reviews.csv'
        # 最大表示ページ数(1,200[20*60]件以降は表示不可)
        self.max_num = max_num
        # 口コミデータフレームの列名
        self.REVIEW_COLUMNS = ['store_id', 'store_name', 'genre', 'rate', 'review_cnt', 
                              'user', 'date', 'rating', 'rating_detail', 'review']
        
        # 出力設定
        self.output_individual_csv = output_individual_csv
        self.output_combined_csv = output_combined_csv
        
        self.review_df = pd.DataFrame(columns=self.REVIEW_COLUMNS)
        
        # DataFrameの場合は辞書のリストに変換
        if isinstance(store_urls_data, pd.DataFrame):
            self.store_data = store_urls_data.to_dict('records')
        else:
            self.store_data = store_urls_data
        
        # 出力ディレクトリを作成
        self._create_output_directories()
        
        self._scrape_reviews()

    def _create_output_directories(self):
        """出力ディレクトリを作成する"""
        if not os.path.exists(self.RESULT_PATH):
            os.makedirs(self.RESULT_PATH)
        
        if self.output_individual_csv and not os.path.exists(self.INDIVIDUAL_CSV_PATH):
            os.makedirs(self.INDIVIDUAL_CSV_PATH)

    def _scrape_reviews(self):
        """口コミ情報をスクレイピングするメソッド"""
        try:
            first_time = time.time()
            
            for store_info in self.store_data:
                start_time = time.time()
                store_id = store_info['store_id']
                store_name = store_info['store_name']
                genre = store_info['genre']
                rate = store_info['rate']
                review_cnt = store_info['review_cnt']
                store_url = store_info['store_url']
                
                print(f"口コミ取得開始: {store_name} (ID: {store_id})")
                
                # 店舗ごとの口コミDataFrameを初期化
                store_review_df = pd.DataFrame(columns=self.REVIEW_COLUMNS)
                
                # ここで review_count を初期化
                review_count = 0  # 実際に取得できた口コミ数をカウント
                
                for page_num in range(1, self.max_num):

                    review_url = store_url + 'dtlrvwlst/COND-0/smp1/D-visit/' + str(page_num) + '/?smp=1&lc=0&rvw_part=all'
                    
                    response = self.connect_url(review_url)
                    # HTML解析用の変数
                    soup = BeautifulSoup(response.text, 'lxml')
                    # エラーページかどうか
                    error_element = soup.find('h2', class_='error-common__title')
                    error = error_element.get_text(strip=True) if error_element else '0'
                    # エラーページの場合はbreak
                    if error == 'お探しのページが見つかりません':
                        break                    

                    print(f"口コミURL: {review_url}")
                    review_url_list = soup.find_all('div', class_='rvw-item js-rvw-item-clickable-area')
    
                    # 各口コミページに遷移し、最新の口コミを取得する
                    review_count = 0  # 実際に取得できた口コミ数をカウント
                    
                    for url in review_url_list:
                        try:
                            # 正しいセレクタで詳細URLを取得
                            detail_link = url.find('a', class_='c-link-circle js-link-bookmark-detail')
                            if detail_link is None:
                                print(f"詳細リンクが見つかりません")
                                continue
                                
                            detail_url = detail_link.get('data-detail-url')
                            if detail_url is None:
                                print(f"data-detail-url属性が見つかりません")
                                continue
                            
                            review_detail_url = 'https://tabelog.com' + detail_url
                            response = self.connect_url(review_detail_url)
                            soup = BeautifulSoup(response.text, 'lxml')
    
                            # 口コミ投稿者
                            user_element = soup.find('a', class_='rvw-item__rvwr-name')
                            user = user_element.get_text(strip=True) if user_element else ""
    
                            # 口コミ訪問月
                            date_element = soup.find('div', class_='rvw-item__single-date')
                            date = date_element.get_text(separator=" ", strip=True) if date_element else ""
                            
                            # 口コミ評価点
                            rating_element = soup.find('b', class_='c-rating-v3__val c-rating-v3__val--strong')
                            rating = rating_element.get_text(strip=True) if rating_element else ""
                            
                            # 口コミ評価点(詳細)
                            rating_detail_element = soup.find('ul', class_='c-rating-detail')
                            rating_detail = rating_detail_element.get_text(separator="/", strip=True) if rating_detail_element else ""
                            
                            # 口コミテキストを取得（見つからない場合は空文字列）
                            review_text = ""
                            # パターン1: rvw-item__rvw-comment内のpタグ
                            review_elements = soup.find_all('div', class_='rvw-item__rvw-comment rvw-item__rvw-comment--custom')
                            if review_elements:
                                for elem in review_elements:
                                    p_tag = elem.find('p')
                                    if p_tag and p_tag.text.strip():
                                        review_text = p_tag.text.strip()
                                        break
                            # パターン2: 他の可能性のあるセレクタ
                            if not review_text:
                                review_elements = soup.find_all('div', class_='rvw-item__rvw-comment--custom')
                                if review_elements:
                                    for elem in review_elements:
                                        p_tag = elem.find('p')
                                        if p_tag and p_tag.text.strip():
                                            review_text = p_tag.text.strip()
                                            break

                            # 口コミテキストがない場合でもログ出力
                            if not review_text:
                                print(f"口コミテキストが見つかりません（空欄として記録）: {review_detail_url}")

                            # 全体のデータフレームに格納（統合CSV用）
                            if self.output_combined_csv:
                                self.add_review_df(store_id, store_name, genre, rate, review_cnt,
                                                 user, date, rating, rating_detail, review_text)
                            
                            # 店舗ごとのデータフレームに格納（個別CSV用）
                            if self.output_individual_csv:
                                store_review_df = self.add_store_review_df(store_review_df, store_id, store_name, genre, rate, review_cnt,
                                                       user, date, rating, rating_detail, review_text)
                            
                            review_count += 1
                                
                        except Exception as e:
                            print(f"口コミ取得中にエラー: {e}")
                            continue
                
                # 店舗ごとのCSVファイルを出力
                if self.output_individual_csv and len(store_review_df) > 0:
                    self.save_store_review_csv(store_id, store_name, store_review_df)
                
                # 結果をログ出力
                if review_count > 0:
                    self.write_review_result(store_name, genre, rate, review_cnt, review_count)
                    process_time = time.time() - start_time
                    print('{}完了：　処理時間：{:.3f}秒, 口コミ取得数：{}件'.format(store_name, process_time, review_count))
                else:
                    print(f"店舗「{store_name}」：口コミが取得できませんでした")

        except requests.exceptions.HTTPError as e:
            print(e)
        except requests.exceptions.ConnectTimeout as e:
            print(e)
        finally:
            end_time = time.time() - first_time
            print('口コミ取得終了、処理時間：{:.3f}秒'.format(end_time))
            
            # 統合CSVファイルを出力
            if self.output_combined_csv and len(self.review_df) > 0:
                combined_csv_path = os.path.join(self.RESULT_PATH, 'reviews_combined.csv')
                self.review_df.to_csv(combined_csv_path, index=False, encoding='utf-8')
                print(f"統合CSV出力完了: {combined_csv_path}")

    def connect_url(self, target_url):
        """対象のURLにアクセスする関数"""
        data = requests.get(target_url, timeout=10)
        data.encoding = data.apparent_encoding
        time.sleep(2)  # アクセス過多を避けるため、2秒スリープ

        if data.status_code == requests.codes.ok:
            return data
        else:
            data.raise_for_status()

    def write_review_result(self, name, genre, rate, count, review_count):
        """口コミ取得結果のログを出力する関数"""
        if not os.path.exists(self.RESULT_PATH):
            os.makedirs(self.RESULT_PATH)
            
        file_path = os.path.join(self.RESULT_PATH, self.REVIEW_RESULT_NAME)
        with open(file_path, mode='a', encoding='utf-8') as f:
            f.write('{}, {}, 評価：{}, 口コミ：{}件, 取得口コミ：{}件\n'.format(name, genre, rate, count, review_count))

    def add_review_df(self, store_id, store_name, genre, rate, review_cnt, user, date, rating, rating_detail, comment):
        """全体の口コミデータフレームに新しい行を追加する関数（統合CSV用）"""
        new_row = {
            'store_id': store_id,
            'store_name': store_name,
            'genre': genre,
            'rate': rate,
            'review_cnt': review_cnt,
            'user': user,
            'date': date,
            'rating': rating,
            'rating_detail': rating_detail, 
            'review': comment
        }
        new_df = pd.DataFrame([new_row])
        self.review_df = pd.concat([self.review_df, new_df], ignore_index=True)
        
    def add_store_review_df(self, store_df, store_id, store_name, genre, rate, review_cnt, user, date, rating, rating_detail, comment):
        """店舗ごとの口コミデータフレームに新しい行を追加する関数（個別CSV用）"""
        new_row = {
            'store_id': store_id,
            'store_name': store_name,
            'genre': genre,
            'rate': rate,
            'review_cnt': review_cnt,
            'user': user,
            'date': date,
            'rating': rating,
            'rating_detail': rating_detail, 
            'review': comment
        }
        new_df = pd.DataFrame([new_row])
        # 元のDataFrameに直接追加（参照を変更）
        updated_df = pd.concat([store_df, new_df], ignore_index=True)
        return updated_df

    def save_store_review_csv(self, store_id, store_name, store_df):
        """店舗ごとの口コミCSVファイルを保存する関数"""
        if len(store_df) > 0:
            # ファイル名に使えない文字を置換
            safe_name = store_name.replace('/', '_').replace('\\', '_').replace(':', '_')
            filename = f"{store_id}_{safe_name}_reviews.csv"
            filepath = os.path.join(self.INDIVIDUAL_CSV_PATH, filename)
            store_df.to_csv(filepath, index=False, encoding='utf-8')
            print(f"個別CSV出力完了: {filepath}")

class SingleStoreDetailExtractor:
    """
    単一の店舗URLから店舗詳細情報を取得するクラス
    """
    def __init__(self, store_url):
        """
        Args:
            store_url (str): 店舗詳細ページのURL
        """
        self.RESULT_PATH = 'out'
        self.STORE_RESULT_NAME = 'store_details_tp.csv'
        self.STORE_COLUMNS = ['store_id', 'store_name', 'genre', 'rate', 'review_cnt', 'store_url',
                             'address', 'latitude', 'longitude', 'business_hours', 'reserve', 
                             'dinner_budget', 'lunch_budget', 'opened']
        
        self.store_url = store_url
        self.store_df = pd.DataFrame(columns=self.STORE_COLUMNS)
        
        self._extract_store_detail()
    
    def _extract_store_detail(self):
        """店舗詳細情報を抽出する"""
        try:
            # URLから店舗IDを抽出
            url_parts = self.store_url.strip('/').split('/')
            shop_id = None
            for part in reversed(url_parts):
                if part.isdigit() and len(part) >= 7:
                    shop_id = part
                    break
            
            if not shop_id:
                print("店舗IDを抽出できませんでした")
                return
            
            # 店舗詳細ページを取得
            response = self.connect_url(self.store_url)
            soup = BeautifulSoup(response.text, 'lxml')
            
            # 店舗名を取得
            store_name_element = soup.find('h2', class_='display-name')
            store_name = store_name_element.get_text(strip=True) if store_name_element else ""
            
            # ジャンルを取得
            genre_element = soup.find('span', class_='rdheader-subinfo__item rdheader-subinfo__item--genre')
            genre = genre_element.get_text(strip=True) if genre_element else ""
            
            # 評価点を取得
            rate_element = soup.find('span', class_='rdheader-rating__score-val-dtl')
            rate = float(rate_element.get_text(strip=True)) if rate_element else 0.0
            
            # 口コミ件数を取得
            review_cnt_element = soup.find('em', class_='num')
            review_cnt = review_cnt_element.get_text(strip=True) if review_cnt_element else "0"
            
            # 詳細情報を取得
            address = self.extract_address(soup)
            business_hours = self.extract_business_hours(soup)
            reserve = self.extract_reserve(soup)
            budget_info = self.extract_budget_info(soup)
            dinner_budget = budget_info.get('dinner', '')
            lunch_budget = budget_info.get('lunch', '')
            opened = self.extract_opened(soup)
            map_info = self.extract_map_info(soup)
            latitude = map_info.get('latitude', '')
            longitude = map_info.get('longitude', '')
            
            # データフレームに追加
            new_row = {
                'store_id': shop_id,
                'store_name': store_name,
                'genre': genre,
                'rate': rate,
                'review_cnt': review_cnt,
                'store_url': self.store_url,
                'address': address,
                'latitude': latitude,
                'longitude': longitude,
                'business_hours': business_hours,
                'reserve': reserve,
                'dinner_budget': dinner_budget,
                'lunch_budget': lunch_budget,
                'opened': opened
            }
            
            self.store_df = pd.DataFrame([new_row])
            
            # 結果を保存
            if not os.path.exists(self.RESULT_PATH):
                os.makedirs(self.RESULT_PATH)
            
            file_path = os.path.join(self.RESULT_PATH, self.STORE_RESULT_NAME)
            self.store_df.to_csv(file_path, index=False, encoding='utf-8')
            
            print(f"店舗情報取得完了: {store_name} (ID: {shop_id})")
            print(f"保存先: {file_path}")
            
        except Exception as e:
            print(f"店舗情報取得エラー: {e}")
    
    def connect_url(self, target_url):
        """対象のURLにアクセスする関数"""
        data = requests.get(target_url, timeout=10)
        data.encoding = data.apparent_encoding
        time.sleep(2)
        
        if data.status_code == requests.codes.ok:
            return data
        else:
            data.raise_for_status()
    
    def extract_address(self, soup):
        """住所情報を抽出"""
        try:
            address_element = soup.find('p', class_='rstinfo-table__address')
            if not address_element:
                return ""
            full_address = address_element.get_text(separator="/", strip=True)
            return full_address
        except Exception as e:
            print(f"住所抽出エラー: {e}")
            return ""
    
    def extract_business_hours(self, soup):
        """営業時間情報を抽出"""
        try:
            business_lists = soup.find_all('ul', class_='rstinfo-table__business-list')
            if not business_lists:
                return ""
            
            all_text_parts = []
            for ul in business_lists:
                text = ul.get_text(separator='/', strip=True)
                if text:
                    text = re.sub(r'\s+', ' ', text)
                    text = text.strip()
                    if text:
                        all_text_parts.append(text)
            
            result = ' '.join(all_text_parts)
            result = re.sub(r'\s+', ' ', result)
            result = result.strip()
            return result
        except Exception as e:
            print(f"営業時間抽出エラー: {e}")
            return ""
    
    def extract_reserve(self, soup):
        """予約可否情報を抽出"""
        try:
            reserve_element = soup.find('p', class_='rstinfo-table__reserve-status')
            if not reserve_element:
                return ""
            reserve = reserve_element.get_text(strip=True)
            return reserve
        except Exception as e:
            print(f"予約可否抽出エラー: {e}")
            return ""
    
    def extract_budget_info(self, soup):
        """予算情報を抽出"""
        try:
            budget_div = soup.find('div', class_='rdheader-budget')
            if not budget_div:
                return {'dinner': '', 'lunch': ''}
            
            budget_info = {'dinner': '', 'lunch': ''}
            budget_items = budget_div.find_all('p', class_='c-rating-v3')
            
            for item in budget_items:
                time_icon = item.find('i', class_=re.compile(r'c-rating-v3__time'))
                if time_icon:
                    if 'c-rating-v3__time--dinner' in time_icon.get('class', []):
                        time_type = 'dinner'
                    elif 'c-rating-v3__time--lunch' in time_icon.get('class', []):
                        time_type = 'lunch'
                    else:
                        continue
                    
                    price_link = item.find('a', class_='rdheader-budget__price-target')
                    if price_link:
                        budget_info[time_type] = price_link.get_text(strip=True)
                    else:
                        price_span = item.find('span', class_='c-rating-v3__val')
                        if price_span:
                            budget_info[time_type] = price_span.get_text(strip=True)
            
            return budget_info
        except Exception as e:
            print(f"予算情報抽出エラー: {e}")
            return {'dinner': '', 'lunch': ''}
    
    def extract_opened(self, soup):
        """オープン日情報を抽出"""
        try:
            opened_element = soup.find('p', class_='rstinfo-opened-date')
            if not opened_element:
                return ""
            opened = opened_element.get_text(strip=True)
            return opened
        except Exception as e:
            print(f"オープン日抽出エラー: {e}")
            return ""
    
    def extract_map_info(self, soup):
        """座標情報を抽出"""
        try:
            map_info = {'latitude': '', 'longitude': ''}
            
            map_wrap = soup.find('div', class_='rstinfo-table__map-wrap')
            if not map_wrap:
                return map_info
            
            img_element = map_wrap.find('img', class_='rstinfo-table__map-image')
            if img_element:
                map_url = img_element.get('data-lazy-src') or img_element.get('src')
                if map_url:
                    center_match = re.search(r'center=([0-9.-]+),([0-9.-]+)', map_url)
                    if center_match:
                        map_info['latitude'] = center_match.group(1)
                        map_info['longitude'] = center_match.group(2)
                    
                    markers_match = re.search(r'markers=color:red\|([0-9.-]+),([0-9.-]+)', map_url)
                    if markers_match:
                        map_info['latitude'] = markers_match.group(1)
                        map_info['longitude'] = markers_match.group(2)
            
            return map_info
        except Exception as e:
            print(f"地図情報抽出エラー: {e}")
            return {'latitude': '', 'longitude': ''}

### 店舗詳細情報の抽出

In [ ]:
if __name__ == '__main__':   
    # 市町村別URLの入力
    df = pd.read_csv('in/Ishikawa_city_url.csv')  # 市町村URLリスト
    
    # ======= リストで複数の市町村を指定 =========
    target_cities = ["七尾市", "鳳珠郡能登町", "鳳珠郡穴水町"]
    df = df[df['city'].isin(target_cities)]
    # ==========================================
    
    print(f"処理対象の市町村: {df['city'].tolist()}")
    print(f"対象数: {len(df)}\n")
    
    # 出力ディレクトリの作成
    output_base_dir = 'out/by_url'
    if not os.path.exists(output_base_dir):
        os.makedirs(output_base_dir)
    
    # 各URLごとに処理
    for idx, row in df.iterrows():
        city_name = row['city']
        city_url = row['url']
        
        print(f"\n{'='*60}")
        print(f"処理開始: {city_name} ({df['city'].tolist().index(city_name) + 1}/{len(df)})")
        print(f"URL: {city_url}")
        print(f"{'='*60}\n")
        
        # 町別URLへの変換
        extractor = TownUrlExtractor() 
        towns = extractor.process_dataframe(pd.DataFrame([row]), url_column='url')
        
        if not towns:
            print(f"警告: {city_name}のタウンURLが取得できませんでした")
            continue
        
        print(f"{city_name}のタウン数: {len(towns)}")
        
        # 店舗詳細のスクレイピング
        store_scraper = TabelogStoreScraper(url_list=towns)
        
        # URLごとのディレクトリを作成（市町村名をベースに）
        # ファイル名に使えない文字を置換
        safe_city_name = city_name.replace('/', '_').replace('\\', '_').replace(':', '_')
        url_output_dir = os.path.join(output_base_dir, safe_city_name)
        if not os.path.exists(url_output_dir):
            os.makedirs(url_output_dir)
        
        # CSVファイル名を生成（市町村名 + タイムスタンプ）
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        csv_filename = f"{safe_city_name}_store_details_{timestamp}.csv"
        csv_filepath = os.path.join(url_output_dir, csv_filename)
        
        # CSVに保存
        store_scraper.store_df.to_csv(csv_filepath, index=False, encoding='utf-8')
        
        print(f"\n{'='*60}")
        print(f"完了: {city_name}")
        print(f"取得店舗数: {len(store_scraper.store_df)}")
        print(f"保存先: {csv_filepath}")
        print(f"{'='*60}\n")
        
        # 次のURLへ移る前に少し待機（サーバー負荷軽減）
        time.sleep(5)
    
    print("\n全ての処理が完了しました")
    print(f"処理した市町村: {', '.join(df['city'].tolist())}")

### store_detailsをマージした後にStore_urlで重複を削除する

In [13]:
from pathlib import Path
import shutil

# フォルダ内の「store_details」を含むファイルを取得してマージ
source_folder = Path('out')
dest_folder = Path('out/pre_merge_archived')
dest_folder.mkdir(exist_ok=True)

files = list(source_folder.glob('*store_details*'))
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

# 重複削除（カラム名を指定）
df = df.drop_duplicates(subset='store_url', keep='first')

# マージ結果を保存
df.to_csv(source_folder / 'merged_store_details.csv', index=False)

# 元ファイルを移行
for f in files:
    shutil.move(str(f), str(dest_folder / f.name))

#### 特定のURLに使用する場合

In [12]:
if __name__ == "__main__":
    store_url = "https://tabelog.com/ishikawa/A1701/A170101/17014750/"
    extractor = SingleStoreDetailExtractor(store_url)

店舗情報取得完了: ネオバーガー (ID: 17014750)
保存先: out\store_details_tp.csv


### 各店舗における口コミ情報の抽出

In [ ]:
if __name__ == '__main__':
    review_scraper_individual_only = TabelogReviewScraper(
        store_scraper.store_df,      # 店舗詳細出力のdataframeを読み込む 
        max_num = 2,                 # 40件まで読み込む
        output_individual_csv=True,  # 店舗ごとのCSVを出力
        output_combined_csv=False    # 統合CSVは出力しない
    ) 

## 補遺

### 対象市町村に含まれる町丁目リストの作成

In [2]:
if __name__ == '__main__':   
    # 市町村別URLの入力
    df = pd.read_csv('in/Ishikawa_city_url.csv')  # 市町村URLリスト
    df = df[df['city'] == "金沢市"]  # 特定の市町村を対象とする場合はここで絞る
    
    # 町別URLへの変換
    extractor = TownUrlExtractor() 
    towns = extractor.process_dataframe(df, url_column='url')
    
    # 出力させる場合
    # extractor.save_results(towns)

### 各町丁目に含まれるすべての店舗の詳細情報の抽出

In [3]:
if __name__ == '__main__':
    store_scraper = TabelogStoreScraper(url_list=towns)
    store_scraper.store_df.to_csv('out/store_details.csv', index=False, encoding='utf-8')

店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17001392/


C:\Users\tachibana\AppData\Local\Temp\ipykernel_22536\3324656195.py:425: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.store_df = pd.concat([self.store_df, new_df], ignore_index=True)


店舗ID: 17001392, 1件目完了：　処理時間：2.925秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17000911/
店舗ID: 17000911, 2件目完了：　処理時間：2.953秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17004576/
店舗ID: 17004576, 3件目完了：　処理時間：2.940秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17004296/
店舗ID: 17004296, 4件目完了：　処理時間：2.892秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17000046/
店舗ID: 17000046, 5件目完了：　処理時間：2.908秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17000047/
店舗ID: 17000047, 6件目完了：　処理時間：2.797秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17005471/
店舗ID: 17005471, 7件目完了：　処理時間：2.825秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17004297/
店舗ID: 17004297, 8件目完了：　処理時間：2.851秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17009851/
店舗ID: 17009851, 9件目完了：　処理時間：2.850秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17006997/
店舗ID: 17006997, 10件目完了：　処理時間：2.707秒
店舗詳細URL: https://tabelog.com/ishikawa/A1701/A170101/17009254/
店舗ID: 17009254, 11件目完了：　処理時間：